In [ ]:
## LOOKING AT THE FIRMS


## DETERMINING THEIR STRENGTH IN THEIR INDUSTRY, OVERALL, AND THE KEY COMPONENTS THAT 
##    MAKE THEM POISED FOR SOLID GROWTH



In [ ]:
#############################  NLP DRIVE  ##################################



In [ ]:
###########################  BUFFET ENGINE  ################################

def engineBuffetLookup(ticker):
    symbol = yfinance.Ticker(ticker).info['symbol']
    shortName = yfinance.Ticker(ticker).info['shortName']
    country = yfinance.Ticker(ticker).info['country']
    state = yfinance.Ticker(ticker).info['state']
    industry = yfinance.Ticker(ticker).info['industry']
    sector = yfinance.Ticker(ticker).info['sector']
    employees = yfinance.Ticker(ticker).info['fullTimeEmployees']
    volume = yfinance.Ticker(ticker).info['averageDailyVolume10Day']
    sharesOutstanding = yfinance.Ticker(ticker).info['sharesOutstanding']
    marketCap = yfinance.Ticker(ticker).info['marketCap']
    heldPercentInstitutions = yfinance.Ticker(ticker).info['heldPercentInstitutions']
    heldPercentInsiders = yfinance.Ticker(ticker).info['heldPercentInsiders']
    enterpriseValue = yfinance.Ticker(ticker).info['enterpriseValue']
    enterpriseToRevenue = yfinance.Ticker(ticker).info['enterpriseToRevenue']
    enterpriseToEbitda = yfinance.Ticker(ticker).info['enterpriseToEbitda']
    forwardEps = yfinance.Ticker(ticker).info['forwardEps']
    trailingEps = yfinance.Ticker(ticker).info['trailingEps']
    bookValue = yfinance.Ticker(ticker).info['bookValue']
    priceToBook = yfinance.Ticker(ticker).info['priceToBook']
    earningsQuarterlyGrowth = yfinance.Ticker(ticker).info['earningsQuarterlyGrowth']
    return symbol, shortName, country, state, industry, sector, employees, volume, sharesOutstanding, marketCap, heldPercentInstitutions, heldPercentInsiders, enterpriseValue, enterpriseToRevenue, enterpriseToEbitda, forwardEps, trailingEps, bookValue, priceToBook, earningsQuarterlyGrowth

In [ ]:
############################  BUILD PILOT  ##################################


##############################################################################
##  INITIAL IMPORTS & SETUP

from datetime import datetime
import yfinance
import sqlite3
import pandas as pd
import csv
yes = ["Yes","yes","ye","y","Y","1"]
no = ["No","no","na","n","N","0","-1"]

pathToCompaniesCSV = 'C:/Users/User/partytime/companiesnyse.csv'
#pathToCompaniesCSV = 'C:/Users/joeca/JNN/companiesnyse.csv'


##############################################################################


#############################################################################
##  DATA POPULATION FUNCTIONS DICTIONARY

popFunctions = {}
def POPbuffet1():
    tableName = 'buffet1'
    fieldNames = ["symbol", "shortName", "country",
                  "state", "industry", "sector", "fullTimeEmployees",
                  "averageDailyVolume10Day", "sharesOutstanding",
                  "marketCap", "heldPercentInstitutions", "heldPercentInsiders",
                  "enterpriseValue", "enterpriseToRevenue", "enterpriseToEbitda",
                  "forwardEps", "trailingEps", "bookValue", "priceToBook",
                  "earningsQuarterlyGrowth"]
    a = engineBuffetLookup(tickersList[0])
    primary = 1
    fields = []
    i = 0
    try:
        for x in a:
            fieldName = fieldNames[i].upper()
            if type(x) == str:
                fieldType = "text"
            else:
                fieldType = "real"
            fields.append(tuple((str(fieldName),str(fieldType),primary)))
            primary = 0
            i += 1
    except IndexError:
        print("Number of Fields does not match number of Variables.")
    placeTuple = "?," * len(a)
    placeTuple = placeTuple[:-1]
    conn = sqlite3.connect('jnn.db')
    try:
        c = conn.cursor()
        command = 'DROP TABLE ' + tableName
        c.execute(command)
        conn.commit()
    except:
        pass
    try:
        c = conn.cursor()
        command = 'CREATE TABLE '+ tableName +' ('
        primaryKeys = " "
        for field in fields:
            if field[2] == 1:
                primaryKeys += field[0] + ", "
            command += field[0] + " " + field[1] + ", "
        primaryKeys = primaryKeys[1:]
        command += "PRIMARY KEY (" + primaryKeys + "))"
        command = command.replace(', )', ')')
        c.execute(command)
        conn.commit()
    except:
        pass
    for ticker in tickersList:
        try:
            g = engineBuffetLookup(ticker)
            c = conn.cursor()
            command = "INSERT INTO buffet1 VALUES " + placeTuple +")"
            c.execute(command,g)
            conn.commit()
        except:
            print(command)
popFunctions['buffet1'] = POPbuffet1

def POPbuffet2():
    for i in range(7):
        print(i)
    print("Fire Hose Protocol: Test Successful.")
popFunctions['buffet2'] = POPbuffet2


##############################################################################

##############################################################################
##  DATABASE CONSTRUCTION FUNCTIONS

def viewAll(tableName):
    conn = sqlite3.connect('jnn.db')
    c = conn.cursor()
    command = "SELECT * FROM " + tableName
    c.execute(command)
    allData = c.fetchall()
    conn.commit()
    allData = pd.DataFrame(allData)
    print(allData)
            
def clearReplaceTable(tableName):
    conn = sqlite3.connect('jnn.db')
    proceed = input("Are you sure you want to wipe the table " + tableName + "? ")
    while True:
        if proceed in yes:
            print("You will now be prompted to structure the new table.")
            fieldCount = input("How many fields would you like to include?")
            counter = 1
            fields = []
            while counter <= int(fieldCount):
                fieldName = input("Enter the name of Field "+str(counter)).upper()
                fieldType = input("Enter the data type of Field "+str(counter))
                primary = int(input("Type 1 to set "+fieldName+"as Primary Key: "))
                if primary != 1:
                    primary = 0
                fields.append(tuple((fieldName,fieldType,primary)))
                counter += 1
            try:
                c = conn.cursor()
                command = 'DROP TABLE ' + tableName
                c.execute(command)
                conn.commit()
            except:
                pass
            c = conn.cursor()
            command = 'CREATE TABLE '+ tableName +' ('
            primaryKeys = " "
            for field in fields:
                if field[2] == 1:
                    primaryKeys += field[0] + ", "
                command += field[0] + " " + field[1] + ", "
            primaryKeys = primaryKeys[1:]
            command += "PRIMARY KEY (" + primaryKeys + "))"
            command = command.replace(', )', ')')
            c.execute(command)
            conn.commit()
            print("Table "+ tableName +" has been wiped.")
            break
        elif proceed not in no:
            proceed = input("Invalid answer. Are you sure you want to wipe the table "+ tableName +"? ")
        else:
            print("Quitting Module...")
            break
    print("Module Terminated.")

def manuallyAddRows(tableName, x):
    counter = len(x)
    valueTuple = '('
    for a in range(counter):
        valueTuple += '?,'
    valueTuple = valueTuple[:-1] + ')'
    conn = sqlite3.connect('jnn.db')
    c = conn.cursor()
    command = "INSERT INTO " + tableName + " VALUES (" + valueTuple + ")"
    c.execute(command,x)
    conn.commit()

        
##############################################################################
##  DATABASE UPDATING FUNCTIONS

def updateCompaniesCSV():
    import urllib
    urllib.request.urlretrieve('https://dumbstockapi.com/stock?format=csv&countries=US', pathToCompaniesCSV)
    
def populateTable(tableName):
    proceed = input("Commence populating table: "+tableName+"?")
    while True:
        if proceed in yes:
            pop = popFunctions[tableName]
            pop()
            print("Data Population Successful.")
            break
        elif proceed not in no:
            proceed = input("Invalid answer. Commence populating table: "+tableName+"?")
        else:
            print("Aborting Data Population...")
            break
    
def FireHoseProtocol(passcode):
    if str(passcode) == str(915):
        proceed = input("Are you sure you want to activate Fire Hose Protocol?")
        startTime = datetime.now()
        updateCompaniesCSV()
        while True:
            if proceed in yes:
                print("Fire Hose Protocol: Activated.")
                for pop in popFunctions.values():
                    pop()
                print("Fire Hose Protocol: Data Population Successful.")
                break
            elif proceed not in no:
                proceed = input("Invalid answer. Are you sure you want to activate Fire Hose Protocol?")
            else:
                print("Aborting Fire Hose Protocol...")
                break
        endTime = datetime.now()
        elapsed = endTime - startTime
        print("Elapsed Time: " + str(elapsed))
    else:
        print("Access Denied. Incorrect access code.")    

#updateCompaniesCSV()
#tickersList = []
#with open(pathToCompaniesCSV) as csvfile:
#    df = csv.reader(csvfile)
#    for row in df:
#        tickersList.append(row[0])
tickersList = ["MSFT"]

In [ ]:
FireHoseProtocol(915)

In [ ]:
viewAll("buffet1")

In [6]:
import mysql.connector

conn = mysql.connector.connect(user='jupiter', password='JupiterGroup$42',
                              host='localhost',
                              database='jnn')

c = conn.cursor()
    
f = c.execute("SHOW TABLES")

for x in f:
    print(x)

TypeError: 'NoneType' object is not iterable